In [1]:
import boto3
import pandas as pd
from botocore.exceptions import ClientError
import configparser
import json
import psycopg2

### Describe our cluster type

In [4]:
# read the config file
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))
KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

CLUSTER_TYPE       = config.get("CLUSTER","CLUSTER_TYPE")
NUM_NODES          = config.get("CLUSTER","NUM_NODES")
NODE_TYPE          = config.get("CLUSTER","NODE_TYPE")

CLUSTER_IDENTIFIER = config.get("CLUSTER","CLUSTER_IDENTIFIER")
DB_NAME                 = config.get("CLUSTER","DB_NAME")
DB_USER            = config.get("CLUSTER","DB_USER")
DB_PASSWORD        = config.get("CLUSTER","DB_PASSWORD")
PORT               = config.get("CLUSTER","DB_PORT")

IAM_ROLE_NAME      = config.get("IAM_ROLE", "ARN")

In [6]:
df = pd.DataFrame({"Param":
                  ["CLUSTER_TYPE", "NUM_NODES", "NODE_TYPE", "CLUSTER_IDENTIFIER", 
                   "DB_NAME", "DB_USER", "DB_PASSWORD", "PORT", "IAM_ROLE_NAME"],
              "Value":
                  [CLUSTER_TYPE, NUM_NODES, NODE_TYPE, CLUSTER_IDENTIFIER, DB_NAME,
                   DB_USER, DB_PASSWORD, PORT, IAM_ROLE_NAME]
             })
df

,Param,Value
0,CLUSTER_TYPE,multi-node
1,NUM_NODES,4
2,NODE_TYPE,dc2.large
3,CLUSTER_IDENTIFIER,dwhCluster
4,DB_NAME,urdwh
5,DB_USER,urdwhuser
6,DB_PASSWORD,Passw0rd
7,PORT,5439
8,IAM_ROLE_NAME,URRole


### Connect to Redshift Cluster

In [7]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".
                format(config.get("CLUSTER", "DB_HOST"),config.get("CLUSTER", "DB_NAME"),
                      config.get("CLUSTER", "DB_USER"),config.get("CLUSTER", "DB_PASSWORD"),
                      config.get("CLUSTER", "DB_PORT")))

In [9]:
cur = conn.cursor()

### Check the no of entries in tables

In [2]:
# GET NUMBER OF ROWS IN EACH TABLE
staging_events_noof_entries = ("""
    SELECT COUNT(*) FROM staging_events
""")

staging_songs_noof_entries = ("""
    SELECT COUNT(*) FROM staging_songs
""")

songplays_noof_entries = ("""
    SELECT COUNT(*) FROM songplays
""")

users_noof_entries = ("""
    SELECT COUNT(*) FROM users
""")

songs_noof_entries = ("""
    SELECT COUNT(*) FROM songs
""")

artists_noof_entries = ("""
    SELECT COUNT(*) FROM artists
""")

time_noof_entries = ("""
    SELECT COUNT(*) FROM time
""")

In [3]:
select_number_entries= [staging_events_noof_entries, staging_songs_noof_entries,
                        songplays_noof_entries, users_noof_entries, songs_noof_entries, 
                        artists_noof_entries, time_noof_entries]

In [23]:
output = []
entries = ["staging_events","staging_songs","songplays","users","songs","artists","time"]
for query in select_number_entries:
    cur.execute(query)
    results = cur.fetchone()

    for row in results:
        output.append(row)

In [20]:
result = pd.DataFrame(
    {'Tables': entries,
     'count': output
    })

In [21]:
result

,Tables,count
0,staging_events,8056
1,staging_songs,14896
2,songplays,333
3,users,105
4,songs,14896
5,artists,10025
6,time,333
